In [1]:
import sys
sys.path.insert(0,'/Users/jarad/fake_folder/Python Libraries')
from jb_libraries import *
%matplotlib inline

### From Rox via hipchat:
* Main account is 21000 payroll liabilities and 21995 is a sub account
* In QB > Lists

### To Do
* match what we pay wageworks and what they charge us
* match what we pay wageworks and what we take out of employees checks
* check out account fees, see what's up
* check out latest month of wageworks for employees who've, etc. (do this last)

### To get from WageWorks
* totals from beginning up to and including may 2018
    * accounts total
    * service fee
    * processing fee
    
### Initial audit
* From the beginning up to but not including June 1, 2018    

### Get data from QB

In [103]:
# get data
csv_super_main = pd.read_csv(r'/Users/jarad/fake_folder/Finance/Recurring/WageWorks Audit/Docs/First CSVs/From QB/Copy of WageWorks - From the beginning to June 1, 2018.csv')

# make a copy
csv_main = csv_super_main.copy()

# lowercase
csv_main.columns = [x.lower() for x in csv_main.columns]

# turn into numbers
ls = ['amount','balance']
for col in ls:
    csv_main[col] = csv_main[col].str.replace(',','')
    csv_main[col] = pd.to_numeric(csv_main[col])
    
# make dates    
csv_main['date'] = pd.to_datetime(csv_main['date'])
csv_main['year and month'] = [str(x)[:7] for x in csv_main['date']]    
csv_main['year'] = [str(x)[:4] for x in csv_main['date']]

# drop these cols
for i in range(3):
    csv_main.drop(['unnamed: ' + str(i)],1,inplace = True)
    
# drop all nans
for i in range(2):
    csv_main.dropna(how = 'all', axis = i, inplace = True) 
    
# no date nulls
csv_main = csv_main[csv_main['date'].isnull() == False]    

# fill na
csv_main['name'].fillna(csv_main['memo'], inplace = True)

# lowercase
for col in csv_main.columns:
    try:
        csv_main[col] = [x.lower() for x in csv_main[col]]
    except:
        pass

# drop zero    
csv_main.drop(csv_main[csv_main['amount'] == 0].index, inplace = True)    
    
# make new columns    
csv_main['balance impact'] = np.where(csv_main['amount'] < 0, 'debit','credit')

# restrict by date, get only up until and including may 2018
csv_main = csv_main[csv_main['date'] <= '2018-05-31']

# get dates
min_ = csv_main['date'].min().date()
max_ = csv_main['date'].max().date()

print('data from %s to %s' % (min_,max_))

data from 2014-08-05 to 2018-05-18


### View "splits" column

In [104]:
csv_main['split'].value_counts()

10010 · chase chk - 1937    6857
66010 · taxes                 74
66005 · service fee            2
Name: split, dtype: int64

In [172]:
splits = csv_main.groupby(['split','balance impact'])[['amount']].sum().unstack(1).fillna(0)
splits.columns = splits.columns.droplevel(0)
splits['row net'] = splits.sum(1)
splits.loc['column net'] = splits.sum()
splits.format_(['m0'] * 3)

,Credit,Debit,Row Net
split,,,
10010 · chase chk - 1937,"$361,642","$-395,358","$-33,716"
66005 · service fee,"$8,145",$0,"$8,145"
66010 · taxes,"$12,993","$-12,993",$0
column net,"$382,780","$-408,351","$-25,571"


### Service Fees

In [174]:
fee_main = csv_main[csv_main['name'].str.contains('fee')].copy()
fee_main['amount'].sum()

8145.0

### Create a new "name 02" column 
* The "name" column has either an employee name, or some variation on of the word "WageWorks"
*****
* If the transaction is associated with an employee:
    * credit: money from their paycheck to our account
    * debit: money from our account to their paycheck (refund for cancelled WageWorks, etc.)
*****    
* If the transaction is associated with WageWorks:
    * credit: refund from WageWorks
    * debit: payment to WageWorks

In [105]:
ls = ['2015 service fee adjustment',
      '2016 service fee adjustment',
      'commuter benefits wageworks',
      'wageworks/commuter express']

csv_main['name 02'] = np.where(csv_main['name'].isin(ls), 'wageworks transaction', 'employee transaction')

In [106]:
s = csv_main.groupby(['name 02','balance impact'])[['amount']].sum().unstack(1)
s.columns = s.columns.droplevel(0)
s['row net'] = s.sum(1)
s.loc['column net'] = s.sum()

In [107]:
s.format_(['m0'] * 3)

,Credit,Debit,Row Net
name 02,,,
employee transaction,"$370,686","$-13,089","$357,597"
wageworks transaction,"$12,094","$-395,262","$-383,168"
column net,"$382,780","$-408,351","$-25,571"


### "Name 02" in-depth

In [109]:
ww_main = csv_main[csv_main['name 02'] == 'wageworks transaction']
ww = ww_main.groupby(['year and month','balance impact'])[['amount']].sum().unstack(1).fillna(0)

emp_main = csv_main[csv_main['name 02'] == 'employee transaction']
emp = emp_main.groupby(['year and month','balance impact'])[['amount']].sum().unstack(1).fillna(0)

### Just Jan 2018

In [132]:
ym = '2018-01'

In [131]:
ww_main[ww_main['year and month'] == ym]

,type,date,num,name,memo,split,amount,balance,year and month,year,balance impact,name 02
6754,check,2018-01-05,NaN,commuter benefits wageworks,35 ccd id: 1943351864,10010 · chase chk - 1937,-11131.75,-31640.32,2018-01,2018,debit,wageworks transaction
6755,deposit,2018-01-12,NaN,commuter benefits wageworks,72 ccd id: 1943351864,10010 · chase chk - 1937,121.00,-31519.32,2018-01,2018,credit,wageworks transaction
6846,check,2018-01-25,NaN,wageworks/commuter express,43918 ccd id: 1943351864,10010 · chase chk - 1937,-50.00,-26604.48,2018-01,2018,debit,wageworks transaction


In [144]:
emp_main[emp_main['year and month'] == ym].groupby(['name','balance impact'])[['amount']].sum().unstack(1).sort_values([('amount','credit')]).head()

,amount
balance impact,credit
name,
andrew f. doro,8.30
garrett b. o'keefe,24.18
stephanie a. donaghy,25.16
martin r. rouse,55.38
kenji j. mason,69.24


### Just May 2018

In [149]:
ym = '2018-05'

In [150]:
ww_main[ww_main['year and month'] == ym]

,type,date,num,name,memo,split,amount,balance,year and month,year,balance impact,name 02
7545,check,2018-05-07,NaN,commuter benefits wageworks,18 ccd id: 1943351864,10010 · chase chk - 1937,-10444.44,-30264.82,2018-05,2018,debit,wageworks transaction
7546,deposit,2018-05-10,NaN,commuter benefits wageworks,79 ccd id: 1943351864,10010 · chase chk - 1937,13.00,-30251.82,2018-05,2018,credit,wageworks transaction
7547,deposit,2018-05-17,NaN,commuter benefits wageworks,42 ccd id: 1943351864,10010 · chase chk - 1937,121.00,-30130.82,2018-05,2018,credit,wageworks transaction


In [154]:
emp_main[emp_main['year and month'] == ym].groupby(['name','balance impact'])[['amount']].sum().unstack(1).sort_values([('amount','credit')]).tail()

,amount
balance impact,credit
name,
sebastian crisantos,120.00
michele santomauro,120.00
luis m. pavon,158.20
kristin m. grass,230.76
theodore c. papes,235.38
